<a href="https://colab.research.google.com/github/rohitN04/intro/blob/main/multi_label_image_classifier_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import kagglehub
import os
from PIL import Image
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import shutil
import pandas as pd
import re
import ast
import numpy as np

# Download latest version
path = kagglehub.dataset_download("nahcocz/multi-label-image-captcha")

print("Path to dataset files:", path)
os.chdir(path)

Path to dataset files: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1


In [ ]:
os.listdir()

['yolo11n-cls.pt', 'labels.csv', 'images']

In [ ]:
# to delete the dataset just in case
# if os.path.exists(path):  # Check if the directory exists
#     shutil.rmtree(path)

In [ ]:
# checking the labels csv
df = pd.read_csv("labels.csv")
LABEL_NAMES = [ 'Bicycle', 'Bridge', 'Bus', 'Car', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Stairs', 'Tractors', 'Traffic Light', 'Other' ]
df.head()

,index,label
0,"0,""[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]""",NaN
1,"1,""[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]""",NaN
2,"2,""[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]""",NaN
3,"3,""[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]""",NaN
4,"4,""[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]""",NaN


In [ ]:
pattern = r'["]'

In [ ]:
# converting string data into lst data
df.loc[:, 'index'] = df.loc[:, 'index'].apply(lambda x: re.split(pattern, x)[1])
df.loc[:, 'index'] = df.loc[:, 'index'].apply(lambda x: ast.literal_eval(x))

In [ ]:
def get_label(lst):
  return [LABEL_NAMES[i] for i, value in enumerate(lst) if value == 1]

df['label'] = df['index'].apply(get_label)

In [ ]:
del df['index']

In [ ]:
df

,label
0,[Bicycle]
1,"[Car, Other]"
2,[Bicycle]
3,[Car]
4,[Other]
...,...
9995,[Car]
9996,[Bridge]
9997,[Other]
9998,[Car]


In [ ]:
model = YOLO("yolo11n-cls.pt")


100%|██████████| 5.52M/5.52M [00:00<00:00, 68.6MB/s]


In [ ]:
os.chdir('images')
os.listdir()[0]

'4392.jpeg'

In [ ]:
imgs = os.listdir()
imgs[0]

'4392.jpeg'

In [ ]:
image = Image.open(imgs[0])
width, height = image.size
print(f"width: {width}, height: {height}")

width: 100, height: 100


In [ ]:
img_location = os.getcwd()
img_location

'/root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images'

In [ ]:
# sorting the files and getting their names
image_files = [f for f in os.listdir(img_location) if f.endswith(('.png', '.jpg', '.jpeg'))]
# splitting the datasets
train, test = train_test_split(image_files, test_size=0.2, shuffle=False)
# seprating the image datasets
train_folder = os.path.join(img_location, "train")
test_folder = os.path.join(img_location, "test")

In [ ]:
os.makedirs(train_folder, exist_ok=True)
for label in LABEL_NAMES:
  os.makedirs(os.path.join(train_folder, label), exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
for label in LABEL_NAMES:
  os.makedirs(os.path.join(test_folder, label), exist_ok=True)

In [ ]:
# Loop through training set
for i in range(len(train)):
    labels = df['label'][i]  # Get the list of labels for the image
    img_path = os.path.join(img_location, train[i])  # Original image path

    if not os.path.exists(img_path):
      continue
    # Ensure the image is copied to all label folders
    for label in labels:
        class_folder = os.path.join(train_folder, str(label))
        os.makedirs(class_folder, exist_ok=True)  # Create folder if not exists
        shutil.copy(img_path, os.path.join(class_folder, train[i]))  # Copy instead of move

# Loop through test set
for i in range(len(test)):
    labels = df['label'][len(train) + i]  # Get the list of labels for the test image
    img_path = os.path.join(img_location, test[i])

    if not os.path.exists(img_path):
      continue
    for label in labels:
        class_folder = os.path.join(test_folder, str(label))
        os.makedirs(class_folder, exist_ok=True)
        shutil.copy(img_path, os.path.join(class_folder, test[i]))  # Copy instead of move

In [ ]:
# Loop through files in the folder
for file in os.listdir(img_location):
    file_path = os.path.join(img_location, file)

    # Check if it's a file and has an image extension
    if os.path.isfile(file_path) and file.endswith(('.png', '.jpg', '.jpeg')):
        os.remove(file_path)  # Delete the image

Streaming output truncated to the last 5000 lines.
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/901.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/2558.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/8268.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/78.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/9188.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/698.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/695.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/8054.jpeg
Deleted: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/9931.jpeg
Deleted: /root/.cache/kaggl

In [ ]:
# print(train_folder)
os.listdir()

['train', 'test']

In [ ]:
results = model.train(data = os.getcwd(), epochs = 100, imgsz=100)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images, epochs=100, time=None, patience=100, batch=16, imgsz=100, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

train: Scanning /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/train... 10006 images, 0 corrupt: 100%|██████████| 10006/10006 [00:01<00:00, 5846.29it/s]


train: New cache created: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/train.cache


val: Scanning /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/test... 2497 images, 0 corrupt: 100%|██████████| 2497/2497 [00:00<00:00, 5911.60it/s]

val: New cache created: /root/.cache/kagglehub/datasets/nahcocz/multi-label-image-captcha/versions/1/images/test.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)


...


      Epoch    GPU_mem       loss  Instances       Size


     24/100         0G      2.129          6        128: 100%|██████████| 626/626 [02:07<00:00,  4.92it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:09<00:00,  8.56it/s]

                   all      0.283      0.699



      Epoch    GPU_mem       loss  Instances       Size


     25/100         0G      2.116         16        128:  24%|██▍       | 151/626 [00:30<01:37,  4.88it/s]